<img src="../../img/ICHEC_Logo.png" alt="Drawing" style="width: 500px;"/>

# Exercise 1 

**make sure you have environment 'numba' displayed in the top right corner!**
- Improve the timing for computing this mandel brot set by njiting one of the functions
- submit to Kay using a job submission script
- Calculate the speed up factor

In [1]:
%%writefile mandel_exercise.py

import numpy as np
import math
import time
import numba
from numba import jit, njit


import matplotlib.pyplot as plt


mandel_timings = []

def plot_mandel(mandel):
    fig=plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    ax.axis('off')
    ax.imshow(mandel, cmap='gnuplot')
    plt.savefig('mandel.png')
    
def kernel(zr, zi, cr, ci, radius, num_iters):
    count = 0
    while ((zr*zr + zi*zi) < (radius*radius)) and count < num_iters:
        zr, zi = zr * zr - zi * zi + cr, 2 * zr * zi + ci
        count += 1
    return count

def compute_mandel_py(cr, ci, N, bound, radius=1000.):
    t0 = time.time()
    mandel = np.empty((N, N), dtype=int)
    grid_x = np.linspace(-bound, bound, N)

    for i, x in enumerate(grid_x):
        for j, y in enumerate(grid_x):
            mandel[i,j] = kernel(x, y, cr, ci, radius, N)
    return mandel, time.time() - t0

def python_run():
    kwargs = dict(cr=0.3852, ci=-0.2026,
              N=400,
              bound=1.2)
    print("Using pure Python")
    mandel_func = compute_mandel_py       
    mandel_set = mandel_set, runtime = mandel_func(**kwargs)
    
    print("Mandelbrot set generated in {} seconds".format(runtime))
    plot_mandel(mandel_set)
    mandel_timings.append(runtime)

    
python_run()

kernel_njit = ###

# copy and paste the compute_mandle_py function & rename it 
# change the kernel function used within it




# copy and paste the run function & rename it
# use the new compute mandel function within it 



## run it twice, first one includes compilation time



print(mandel_timings)

# what percentage did it speed up by?
print("speedup factor using njit on the kernel")
print(mandel_timings[0]/mandel_timings[2])

Overwriting mandel_exercise.py


In [2]:
%%writefile mandel_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=mandel
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python mandel_exercise.py

exit 0

Overwriting job.sh


# Exercise 2
Try out `@njit`, eager compilation, cache and fastmath on the Python function below and compare timings.

In [ ]:
%%writefile montecarlo_exercise.py
import numpy as np
import math
import time
import numba
from numba import jit, njit

def pi_montecarlo_python(n):
    in_circle = 0
    for i in range(n):
        x, y = np.random.random(), np.random.random()
        if x ** 2 + y ** 2 <= 1.0:
            in_circle += 1
        
    return 4.0 * in_circle / n

pi_montecarlo_python_njit = ###

pi_montecarlo_python_eag = ###

pi_montecarlo_python_fm = ###

pi_montecarlo_python_cache = ###

pi_montecarlo_python_ALL = ###




n = 10000000

print('python njit including compilation')
%time pi_montecarlo_python_njit(n)
print("")
print('python njit')
%time pi_montecarlo_python_njit(n)
print("")
print('python Eager including compilation')
%time pi_montecarlo_python_eag(n)
print("")
print('python Eager')
%time pi_montecarlo_python_eag(n)
print("")
print('python fastmath including compilation')
%time pi_montecarlo_python_fm(n)
print("")
print('python fastmath')
%time pi_montecarlo_python_fm(n)
print("")
print('python cached including compilation')
%time pi_montecarlo_python_cache(n)
print("")
print('python cached')
%time pi_montecarlo_python_cache(n)
print("")
print('python ALL including compilation')
%time pi_montecarlo_python_ALL(n)
print("")
print('python ALL')
%time pi_montecarlo_python_ALL(n)

In [ ]:
%%writefile montecarlo_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=montecarlo
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python montecarlo_exercise.py

exit 0

# Exercise 3 
- See `is_prime`function below.
- `njit` it, `vectorize` it, and vectorize it with target set to `parallel`.
- Compare all 3 timings

In [ ]:
%%writefile prime_exercise.py

import numpy as np
import math
import time
import numba
from numba import jit, njit

def is_prime(n):
    if n <= 1:
        raise ArithmeticError('n <= 1')
    if n == 2 or n == 3:
        return True
    elif n % 2 == 0:
        return False
    else:
        n_sqrt = math.ceil(math.sqrt(n))
        for i in range(3, n_sqrt):
            if n % i == 0:
                return False

    return True

numbers = np.random.randint(2, 100000, dtype=np.int64, size=1000000)

is_prime_njit = ###
is_prime_v = ###
is_prime_vp = ###


print("Pure Python")
t0 = time.time()
p1 = [is_prime(x) for x in numbers]
print(time.time()-t0)
print("")

print("njit")
t0 = time.time()
p2 = [is_prime_njit(x) for x in numbers]
print(time.time()-t0)

t0 = time.time()
p2 = [is_prime_njit(x) for x in numbers]
print(time.time()-t0)
print("")

print("vectorized")
t0 = time.time()
p3 = is_prime_v(numbers)
print(time.time()-t0)
print("")

print("vectorized and parallel")
t0 = time.time()
p3 = is_prime_vp(numbers)
print(time.time()-t0)

In [ ]:
%%writefile prime_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=prime
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python prime_exercise.py

exit 0


# BIG EXERCISE

**Using what you've learnt play around with the functions that implement the heat equation.**

### Answer the following questions (can use bottle_large.dat - time differences are more apparent)

1) Which version of the evolve function respond better to optimisation by Numba, the Python or the NumPy one?

2) Try eager compilation on the better function - record time
    - Hint: input type for field is `float64[:,:]`
    
3) Try things like `parallel=True`, `prange`, `fastmath=True`. What combination gave the best optimisation?

4) Which loop is better to have the `prange` on?

5) `njit` the iterate function (solve the error first [hint: comment out python object part], does it speed up? 

6) notice anything wrong when you parallelise the iterate function?

In [3]:
%%writefile heat_equation.py
import matplotlib.pyplot as plt
import numpy as np
import time

# Set the colormap
plt.rcParams['image.cmap'] = 'BrBG'

# Basic parameters
a = 0.5                # Diffusion constant
timesteps = 200        # Number of time-steps to evolve system
image_interval = 4000  # Write frequency for png files

# Grid spacings
dx = 0.01
dy = 0.01
dx2 = dx**2
dy2 = dy**2

# For stability, this is the largest interval possible
# for the size of the time-step:
dt = dx2*dy2 / ( 2*a*(dx2+dy2) )

def evolve_python(u, u_previous, a, dt, dx2, dy2):
    
    n = u.shape[0]
    m = u.shape[1]

    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) /dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) /dy2 )
    u_previous[:] = u[:]
    

def evolve_numpy(u, u_previous, a, dt, dx2, dy2):

    del_sqrd_u =((u_previous[:-2, 1:-1] - 2*u_previous[1:-1, 1:-1] + u_previous[2:, 1:-1]) / 
                 dx2 + (u_previous[1:-1, :-2] - 2*u_previous[1:-1, 1:-1] + u_previous[1:-1, 2:]) / dy2) 
    
    u[1:-1,1:-1] = u_previous[1:-1,1:-1] + dt * a *  del_sqrd_u

    u_previous[:,:] = u[:,:]

def initial_fields(filename):
    field = np.loadtxt(filename)
    field0 = np.copy(field)
    return field, field0


def write_field(field, step):
    plt.gca().clear()
    #plt.hold(False)
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def iterate(field, field0, timesteps, image_interval): 
    
    for i in range(1,timesteps+1):
        evolve_numpy(field, field0, a, dt, dx2, dy2)
        if i % image_interval == 0:
            write_field(field, i)

def main():
    field, field0 = initial_fields('bottle_large.dat')

    write_field(field, 0)
    
    iterate(field, field0, timesteps, image_interval)
    
    write_field(field, timesteps)


t0 = time.time()
main()
print(time.time()-t0)


Writing heat_equation.py


In [5]:
%%writefile heat_job.sh
#!/bin/bash
#SBATCH --nodes=1
#SBATCH --time=00:10:00
#SBATCH -A course
#SBATCH --job-name=heat
#SBATCH -p CourseDevQ
#SBATCH --reservation=CourseMay


module purge
module load conda
module list

source activate numba

cd $SLURM_SUBMIT_DIR

python heat_exercise.py

exit 0

Writing heat_job.sh
